In [ ]:
#!pip install requests-html requests

In [1]:
import requests
from requests_html import HTML
import matplotlib
import pandas as pd
import time

In [2]:
def clean_scrapped_data(text, keyname=None):
    if keyname == 'tags':
        return text.replace('\n','-')
    return text

In [3]:
def parse_tagged_page(html):
    question_summaries = html.find(".s-post-summary")
    key_names = ['questions', 'stats', 'tags']
    classes_needed = [".s-link", ".s-post-summary--stats-item-number", ".ml0"]
    key_stats = ['votes', 'answers', 'views']

    datas = []

    for q_el in question_summaries:
        question_data = {}
    
        for i, _class in enumerate(classes_needed):
        
            if _class == '.s-post-summary--stats-item-number':
                for j in range(3):
    #sub_el = this_questions_element.find('.s-post-summary--stats-item-number')[j].text  
                    question_data[key_stats[j]] = q_el.find('.s-post-summary--stats-item-number')[j].text 
            #question_data[key_stats[1]] = sub_el[1]
            #question_data[key_stats[2]] = sub_el[2]

            else:
                sub_el = q_el.find(_class, first=True)
        #print(sub_el.text)
                keyname = key_names[i]
                question_data[keyname] = clean_scrapped_data(sub_el.text, keyname=keyname)
    
        datas.append(question_data)

    return datas    

In [4]:
def extract_data_from_url(url):
    r = requests.get(url)
    if r.status_code not in range(200,299):
        return []
    html_str = r.text
    html = HTML(html=html_str)
    datas = parse_tagged_page(html)

    return datas

In [5]:
def scrape_tag(tag = 'python', query_filter = "Votes", page_size=25, max_pages=50):

    base_url = 'https://stackoverflow.com/questions/tagged/'
    datas = []
    for p in range(max_pages):
        page_num = p + 1
        url = f"{base_url}{tag}?tab={query_filter}&page{page_num}&pagesize={page_size}"
        datas += extract_data_from_url(url)
        time.sleep(1.5)

    return datas

In [6]:
datas = scrape_tag(tag='python')

In [7]:
len(datas)

2500

In [8]:
df = pd.DataFrame(datas)
df.head()

,questions,votes,answers,views,tags
0,"What does the ""yield"" keyword do in Python?",12528,51,3.2m,python-iterator-generator
1,"What does if __name__ == ""__main__"": do?",7948,45,4.4m,python-namespaces-program-entry-point-python-m...
2,Does Python have a ternary conditional operator?,7660,31,2.7m,python-operators-conditional-operator
3,What are metaclasses in Python?,7149,25,1.1m,python-oop-metaclass-python-class-python-datam...
4,How do I check whether a file exists without e...,6922,40,5.1m,python-file-file-exists


In [9]:
df.to_csv('Stackoverflow-Python.csv', index=False)

### Resources:

This projects was inspired by the 30 Days of Python - Day 22 challenges correcting the code to the Stackoverflow current structure. 

Youtube video-link: https://www.youtube.com/watch?v=BFAQCDr6Qvc&list=PLEsfXFp6DpzQjDBvhNy5YbaBx9j-ZsUe6&index=24